In [1]:
import sys
sys.path.append("/Users/PRVATE/models-master/")

In [2]:
import collections
import itertools
import random

# Import libraries
from absl import app
from absl import flags
from absl import logging
import tensorflow as tf

from official.nlp.bert import tokenization

In [ ]:
# Only for jupyter notebook
app.flags.DEFINE_string('f', '', 'kernel')

In [3]:

flags.DEFINE_string("input_file", None,
                    "Input raw text file (or comma-separated list of files).")

flags.DEFINE_string(
    "output_file", None,
    "Output TF example file (or comma-separated list of files).")

flags.DEFINE_string("vocab_file", None,
                    "The vocabulary file that the BERT model was trained on.")

flags.DEFINE_bool(
    "do_lower_case", True,
    "Whether to lower case the input text. Should be True for uncased "
    "models and False for cased models.")

flags.DEFINE_bool(
    "do_whole_word_mask", False,
    "Whether to use whole word masking rather than per-WordPiece masking.")

flags.DEFINE_integer(
    "max_ngram_size", None,
    "Mask contiguous whole words (n-grams) of up to `max_ngram_size` using a "
    "weighting scheme to favor shorter n-grams. "
    "Note: `--do_whole_word_mask=True` must also be set when n-gram masking.")

flags.DEFINE_bool(
    "gzip_compress", False,
    "Whether to use `GZIP` compress option to get compressed TFRecord files.")

flags.DEFINE_bool(
    "use_v2_feature_names", False,
    "Whether to use the feature names consistent with the models.")

flags.DEFINE_integer("max_seq_length", 128, "Maximum sequence length.")

flags.DEFINE_integer("max_predictions_per_seq", 20,
                     "Maximum number of masked LM predictions per sequence.")

flags.DEFINE_integer("random_seed", 12345, "Random seed for data generation.")

flags.DEFINE_integer(
    "dupe_factor", 10,
    "Number of times to duplicate the input data (with different masks).")

flags.DEFINE_float("masked_lm_prob", 0.15, "Masked LM probability.")

flags.DEFINE_float(
    "short_seq_prob", 0.1,
    "Probability of creating sequences which are shorter than the "
    "maximum length.")

In [15]:
def main(_):
    pass
FLAGS = flags.FLAGS
app.run(main)

SystemExit: 

/Users/PRVATE/anaconda3/envs/venv_tf_nightly/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [19]:
max_seq_length = FLAGS.max_seq_length
dupe_factor = FLAGS.dupe_factor
short_seq_prob = FLAGS.short_seq_prob
masked_lm_prob = FLAGS.masked_lm_prob
max_predictions_per_seq = FLAGS.max_predictions_per_seq
rng =  random.Random(FLAGS.random_seed)
do_whole_word_mask=False
max_ngram_size=None

In [21]:
vocab_file = '/Users/PRVATE/pretrained_models/bert_uncased/assets/vocab.txt'
tokenizer  = tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=True)
vocab_words = list(tokenizer.vocab.keys())

In [107]:
def truncate_seq_pair(tokens_a, tokens_b, max_num_tokens, rng):
  """Truncates a pair of sequences to a maximum sequence length."""
  while True:
    total_length = len(tokens_a) + len(tokens_b)
    if total_length <= max_num_tokens:
      break

    trunc_tokens = tokens_a if len(tokens_a) > len(tokens_b) else tokens_b
    assert len(trunc_tokens) >= 1

    # We want to sometimes truncate from the front and sometimes from the
    # back to add more randomness and avoid biases.
    if rng.random() < 0.5:
      del trunc_tokens[0]
    else:
      trunc_tokens.pop()
    
def _window(iterable, size):
  """Helper to create a sliding window iterator with a given size.
  E.g.,
    input = [1, 2, 3, 4]
    _window(input, 1) => [1], [2], [3], [4]
    _window(input, 2) => [1, 2], [2, 3], [3, 4]
    _window(input, 3) => [1, 2, 3], [2, 3, 4]
    _window(input, 4) => [1, 2, 3, 4]
    _window(input, 5) => None
  Arguments:
    iterable: elements to iterate over.
    size: size of the window.
  Yields:
    Elements of `iterable` batched into a sliding window of length `size`.
  """
  i = iter(iterable)
  window = []
  try:
    for e in range(0, size):
      window.append(next(i))
    yield window
  except StopIteration:
    # handle the case where iterable's length is less than the window size.
    return
  for e in i:
    window = window[1:] + [e]
    yield window
    
def _contiguous(sorted_grams):
  """Test whether a sequence of grams is contiguous.
  Arguments:
    sorted_grams: _Grams which are sorted in increasing order.
  Returns:
    True if `sorted_grams` are touching each other.
  E.g.,
    _contiguous([(1, 4), (4, 5), (5, 10)]) == True
    _contiguous([(1, 2), (4, 5)]) == False
  """
  for a, b in _window(sorted_grams, 2):
    if a.end != b.begin:
      return False
  return True

In [49]:
line = 'Family is love and love is happiness. Lets add a word in malayalam'

input_files = ['input.txt']
all_documents = [[]]
for input_file in input_files:
    with tf.io.gfile.GFile(input_file, "rb") as reader:
      while True:
        line = tokenization.convert_to_unicode(reader.readline())
        if not line:
          break
        line = line.strip()

        # Empty lines are used as document delimiters
        if not line:
          all_documents.append([])
        tokens = tokenizer.tokenize(line)
        if tokens:
          all_documents[-1].append(tokens)
        
all_documents = [x for x in all_documents if x]

In [51]:
document_index = 0

In [52]:
document = all_documents[document_index]
# Account for [CLS], [SEP], [SEP]
max_num_tokens = max_seq_length - 3
target_seq_length = max_num_tokens
if rng.random() < short_seq_prob:
    target_seq_length = rng.randint(2, max_num_tokens)

In [53]:
short_seq_prob

0.1

In [61]:
instances = []
current_chunk = []
current_length = 0
i = 0

In [62]:
len(document)

3

In [63]:
segment = document[i]
current_chunk.append(segment)
current_length += len(segment)

In [69]:
a_end = 1

In [70]:
tokens_a = []
for j in range(a_end):
    tokens_a.extend(current_chunk[j])

In [72]:
        tokens_b = []
        # Random next
        is_random_next = False
        if len(current_chunk) == 1 or rng.random() < 0.5:
          is_random_next = True
          target_b_length = target_seq_length - len(tokens_a)

          # This should rarely go for more than one iteration for large
          # corpora. However, just to be careful, we try to make sure that
          # the random document is not the same as the document
          # we're processing.
          for _ in range(10):
            random_document_index = rng.randint(0, len(all_documents) - 1)
            if random_document_index != document_index:
              break

          random_document = all_documents[random_document_index]
          random_start = rng.randint(0, len(random_document) - 1)
          for j in range(random_start, len(random_document)):
            tokens_b.extend(random_document[j])
            if len(tokens_b) >= target_b_length:
              break
          # We didn't actually use these segments so we "put them back" so
          # they don't go to waste.
          num_unused_segments = len(current_chunk) - a_end
          i -= num_unused_segments
        # Actual next
        else:
          is_random_next = False
          for j in range(a_end, len(current_chunk)):
            tokens_b.extend(current_chunk[j])

In [79]:
tokens_a

['this',
 'text',
 'is',
 'included',
 'to',
 'make',
 'sure',
 'unicode',
 'is',
 'handled',
 'properly',
 ':',
 '力',
 '加',
 '勝',
 '北',
 '区',
 'ᴵ',
 '##ᴺ',
 '##ᵀ',
 '##ᵃ',
 '##ছ',
 '##জ',
 '##ট',
 '##ড',
 '##ণ',
 '##ত']

In [80]:
tokens_b

['this',
 'sample',
 'text',
 'is',
 'public',
 'domain',
 'and',
 'was',
 'randomly',
 'selected',
 'from',
 'project',
 'gut',
 '##tenberg',
 '.']

In [81]:
truncate_seq_pair(tokens_a, tokens_b, max_num_tokens, rng)

In [84]:
tokens = []
segment_ids = []
tokens.append("[CLS]")
segment_ids.append(0)
for token in tokens_a:
  tokens.append(token)
  segment_ids.append(0)

tokens.append("[SEP]")
segment_ids.append(0)

for token in tokens_b:
  tokens.append(token)
  segment_ids.append(1)
tokens.append("[SEP]")
segment_ids.append(1)

In [143]:
# A _Gram is a [half-open) interval of token indices which form a word.
# E.g.,
#   words:  ["The", "doghouse"]
#   tokens: ["The", "dog", "##house"]
#   grams:  [(0,1), (1,3)]
_Gram = collections.namedtuple("_Gram", ["begin", "end"])
max_ngram_size = 1

In [144]:
grams = [_Gram(i, i+1) for i in range(0, len(tokens))
             if tokens[i] not in ["[CLS]", "[SEP]"]]
num_to_predict = min(max_predictions_per_seq,
                       max(1, int(round(len(tokens) * masked_lm_prob))))
# Generate masks.  If `max_ngram_size` in [0, None] it means we're doing
# whole word masking or token level masking.  Both of these can be treated
# as the `max_ngram_size=1` case.

# masked_grams = _masking_ngrams(grams, max_ngram_size or 1,
#                              num_to_predict, rng)

In [145]:
grams

[_Gram(begin=1, end=2),
 _Gram(begin=2, end=3),
 _Gram(begin=3, end=4),
 _Gram(begin=4, end=5),
 _Gram(begin=5, end=6),
 _Gram(begin=6, end=7),
 _Gram(begin=7, end=8),
 _Gram(begin=8, end=9),
 _Gram(begin=9, end=10),
 _Gram(begin=10, end=11),
 _Gram(begin=11, end=12),
 _Gram(begin=12, end=13),
 _Gram(begin=13, end=14),
 _Gram(begin=14, end=15),
 _Gram(begin=15, end=16),
 _Gram(begin=16, end=17),
 _Gram(begin=17, end=18),
 _Gram(begin=18, end=19),
 _Gram(begin=19, end=20),
 _Gram(begin=20, end=21),
 _Gram(begin=21, end=22),
 _Gram(begin=22, end=23),
 _Gram(begin=23, end=24),
 _Gram(begin=24, end=25),
 _Gram(begin=25, end=26),
 _Gram(begin=26, end=27),
 _Gram(begin=27, end=28),
 _Gram(begin=29, end=30),
 _Gram(begin=30, end=31),
 _Gram(begin=31, end=32),
 _Gram(begin=32, end=33),
 _Gram(begin=33, end=34),
 _Gram(begin=34, end=35),
 _Gram(begin=35, end=36),
 _Gram(begin=36, end=37),
 _Gram(begin=37, end=38),
 _Gram(begin=38, end=39),
 _Gram(begin=39, end=40),
 _Gram(begin=40, end=41),
 _G

In [146]:

  grams = sorted(grams)
  num_tokens = grams[-1].end

  # Ensure our grams are valid (i.e., they don't overlap).
  for a, b in _window(grams, 2):
    if a.end > b.begin:
      raise ValueError("overlapping grams: {}".format(grams))

  # Build map from n-gram length to list of n-grams.
  ngrams = {i: [] for i in range(1, max_ngram_size+1)}

In [147]:
  # Build map from n-gram length to list of n-grams.
  ngrams = {i: [] for i in range(1, max_ngram_size+1)}
  for gram_size in range(1, max_ngram_size+1):
    for g in _window(grams, gram_size):
      if _contiguous(g):
        # Add an n-gram which spans these one-grams.
        ngrams[gram_size].append(_Gram(g[0].begin, g[-1].end))

In [148]:
ngrams

{1: [_Gram(begin=1, end=2),
  _Gram(begin=2, end=3),
  _Gram(begin=3, end=4),
  _Gram(begin=4, end=5),
  _Gram(begin=5, end=6),
  _Gram(begin=6, end=7),
  _Gram(begin=7, end=8),
  _Gram(begin=8, end=9),
  _Gram(begin=9, end=10),
  _Gram(begin=10, end=11),
  _Gram(begin=11, end=12),
  _Gram(begin=12, end=13),
  _Gram(begin=13, end=14),
  _Gram(begin=14, end=15),
  _Gram(begin=15, end=16),
  _Gram(begin=16, end=17),
  _Gram(begin=17, end=18),
  _Gram(begin=18, end=19),
  _Gram(begin=19, end=20),
  _Gram(begin=20, end=21),
  _Gram(begin=21, end=22),
  _Gram(begin=22, end=23),
  _Gram(begin=23, end=24),
  _Gram(begin=24, end=25),
  _Gram(begin=25, end=26),
  _Gram(begin=26, end=27),
  _Gram(begin=27, end=28),
  _Gram(begin=29, end=30),
  _Gram(begin=30, end=31),
  _Gram(begin=31, end=32),
  _Gram(begin=32, end=33),
  _Gram(begin=33, end=34),
  _Gram(begin=34, end=35),
  _Gram(begin=35, end=36),
  _Gram(begin=36, end=37),
  _Gram(begin=37, end=38),
  _Gram(begin=38, end=39),
  _Gram(begin=3

In [149]:
  # Create the weighting for n-gram length selection.
  # Stored cummulatively for `random.choices` below.
  cummulative_weights = list(
      itertools.accumulate([1./n for n in range(1, max_ngram_size+1)]))

In [150]:
cummulative_weights

[1.0]

In [151]:
  output_ngrams = []
  max_masked_tokens = num_to_predict
  masked_tokens = [False] * num_tokens
  while (sum(masked_tokens) < max_masked_tokens and
         sum(len(s) for s in ngrams.values())):
    # Pick an n-gram size based on our weights.
    sz = random.choices(range(1, max_ngram_size+1),
                        cum_weights=cummulative_weights)[0]

    # Ensure this size doesn't result in too many masked tokens.
    # E.g., a two-gram contains _at least_ two tokens.
    if sum(masked_tokens) + sz > max_masked_tokens:
      # All n-grams of this length are too long and can be removed from
      # consideration.
      ngrams[sz].clear()
      continue

    # All of the n-grams of this size have been used.
    if not ngrams[sz]:
      continue

    # Choose a random n-gram of the given size.
    gram = ngrams[sz].pop()
    num_gram_tokens = gram.end-gram.begin

    # Check if this would add too many tokens.
    if num_gram_tokens + sum(masked_tokens) > max_masked_tokens:
      continue

    # Check if any of the tokens in this gram have already been masked.
    if sum(masked_tokens[gram.begin:gram.end]):
      continue

    # Found a usable n-gram!  Mark its tokens as masked and add it to return.
    masked_tokens[gram.begin:gram.end] = [True] * (gram.end-gram.begin)
    output_ngrams.append(gram)

In [152]:
  MaskedLmInstance = collections.namedtuple("MaskedLmInstance",
                                          ["index", "label"])
  masked_grams = output_ngrams
  masked_lms = []
  output_tokens = list(tokens)
  for gram in masked_grams:
    # 80% of the time, replace all n-gram tokens with [MASK]
    if rng.random() < 0.8:
      replacement_action = lambda idx: "[MASK]"
    else:
      # 10% of the time, keep all the original n-gram tokens.
      if rng.random() < 0.5:
        replacement_action = lambda idx: tokens[idx]
      # 10% of the time, replace each n-gram token with a random word.
      else:
        replacement_action = lambda idx: rng.choice(vocab_words)

    for idx in range(gram.begin, gram.end):
      output_tokens[idx] = replacement_action(idx)
      masked_lms.append(MaskedLmInstance(index=idx, label=tokens[idx]))

In [153]:
masked_lms

[MaskedLmInstance(index=43, label='.'),
 MaskedLmInstance(index=42, label='##tenberg'),
 MaskedLmInstance(index=41, label='gut'),
 MaskedLmInstance(index=40, label='project'),
 MaskedLmInstance(index=39, label='from'),
 MaskedLmInstance(index=38, label='selected'),
 MaskedLmInstance(index=37, label='randomly')]

In [154]:
  assert len(masked_lms) <= num_to_predict
  masked_lms = sorted(masked_lms, key=lambda x: x.index)

In [155]:
masked_lms

[MaskedLmInstance(index=37, label='randomly'),
 MaskedLmInstance(index=38, label='selected'),
 MaskedLmInstance(index=39, label='from'),
 MaskedLmInstance(index=40, label='project'),
 MaskedLmInstance(index=41, label='gut'),
 MaskedLmInstance(index=42, label='##tenberg'),
 MaskedLmInstance(index=43, label='.')]

In [156]:
masked_grams

[_Gram(begin=43, end=44),
 _Gram(begin=42, end=43),
 _Gram(begin=41, end=42),
 _Gram(begin=40, end=41),
 _Gram(begin=39, end=40),
 _Gram(begin=38, end=39),
 _Gram(begin=37, end=38)]

In [171]:
tokens

['[CLS]',
 'this',
 'text',
 'is',
 'included',
 'to',
 'make',
 'sure',
 'unicode',
 'is',
 'handled',
 'properly',
 ':',
 '力',
 '加',
 '勝',
 '北',
 '区',
 'ᴵ',
 '##ᴺ',
 '##ᵀ',
 '##ᵃ',
 '##ছ',
 '##জ',
 '##ট',
 '##ড',
 '##ণ',
 '##ত',
 '[SEP]',
 'this',
 'sample',
 'text',
 'is',
 'public',
 'domain',
 'and',
 'was',
 'randomly',
 'selected',
 'from',
 'project',
 'gut',
 '##tenberg',
 '.',
 '[SEP]']

In [157]:
  cand_indexes = []
  for (i, token) in enumerate(tokens):
    if token == "[CLS]" or token == "[SEP]":
      continue
    # Whole Word Masking means that if we mask all of the wordpieces
    # corresponding to an original word. When a word has been split into
    # WordPieces, the first token does not have any marker and any subsequence
    # tokens are prefixed with ##. So whenever we see the ## token, we
    # append it to the previous set of word indexes.
    #
    # Note that Whole Word Masking does *not* change the training code
    # at all -- we still predict each WordPiece independently, softmaxed
    # over the entire vocabulary.
    if (do_whole_word_mask and len(cand_indexes) >= 1 and
        token.startswith("##")):
      cand_indexes[-1].append(i)
    else:
      cand_indexes.append([i])

In [164]:
  output_tokens = list(tokens)

  num_to_predict = min(max_predictions_per_seq,
                       max(1, int(round(len(tokens) * masked_lm_prob))))

  masked_lms = []
  covered_indexes = set()
  for index_set in cand_indexes:
    if len(masked_lms) >= num_to_predict:
      break
    # If adding a whole-word mask would exceed the maximum number of
    # predictions, then just skip this candidate.
    if len(masked_lms) + len(index_set) > num_to_predict:
      continue
    is_any_index_covered = False
    for index in index_set:
      if index in covered_indexes:
        is_any_index_covered = True
        break
    if is_any_index_covered:
      continue
    for index in index_set:
      covered_indexes.add(index)

      masked_token = None
      # 80% of the time, replace with [MASK]
      if rng.random() < 0.8:
        masked_token = "[MASK]"
      else:
        # 10% of the time, keep original
        if rng.random() < 0.5:
          masked_token = tokens[index]
        # 10% of the time, replace with random word
        else:
          masked_token = vocab_words[rng.randint(0, len(vocab_words) - 1)]
          print("masked token", masked_token)

      output_tokens[index] = masked_token

      masked_lms.append(MaskedLmInstance(index=index, label=tokens[index]))

masked token donnell


In [166]:
masked_lms

[MaskedLmInstance(index=1, label='this'),
 MaskedLmInstance(index=2, label='text'),
 MaskedLmInstance(index=3, label='is'),
 MaskedLmInstance(index=4, label='included'),
 MaskedLmInstance(index=5, label='to'),
 MaskedLmInstance(index=6, label='make'),
 MaskedLmInstance(index=7, label='sure')]

In [167]:
output_tokens

['[CLS]',
 '[MASK]',
 '[MASK]',
 '[MASK]',
 'donnell',
 '[MASK]',
 '[MASK]',
 '[MASK]',
 'unicode',
 'is',
 'handled',
 'properly',
 ':',
 '力',
 '加',
 '勝',
 '北',
 '区',
 'ᴵ',
 '##ᴺ',
 '##ᵀ',
 '##ᵃ',
 '##ছ',
 '##জ',
 '##ট',
 '##ড',
 '##ণ',
 '##ত',
 '[SEP]',
 'this',
 'sample',
 'text',
 'is',
 'public',
 'domain',
 'and',
 'was',
 'randomly',
 'selected',
 'from',
 'project',
 'gut',
 '##tenberg',
 '.',
 '[SEP]']

In [168]:
  assert len(masked_lms) <= num_to_predict
  masked_lms = sorted(masked_lms, key=lambda x: x.index)

  masked_lm_positions = []
  masked_lm_labels = []
  for p in masked_lms:
    masked_lm_positions.append(p.index)
    masked_lm_labels.append(p.label)

In [169]:
masked_lm_positions

[1, 2, 3, 4, 5, 6, 7]

In [170]:
masked_lm_labels

['this', 'text', 'is', 'included', 'to', 'make', 'sure']